<a href="https://colab.research.google.com/github/natsumiA/Fashion_MNIST/blob/main/keras_gridsearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [170]:
!pip install scikeras

In [171]:
import keras
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier
from keras import backend as K
from sklearn.model_selection import GridSearchCV, KFold

In [172]:
num_classes = 10
img_rows, img_cols = 28, 28
image_size = 28

In [173]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [174]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test .reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [175]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /=255

In [176]:
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [177]:
input_shape

(28, 28, 1)

In [178]:
def make_model(num_layer, activation, mid_units, num_filters, dropout_rate, optimizer):

    model = Sequential()
    model.add(Conv2D(int(num_filters[0]), 3, padding='same', input_shape=input_shape))
    model.add(Activation(activation))
    
    for i in range(1, num_layer):
        model.add(Conv2D(int(num_filters[i]), 3))
        model.add(Activation(activation))
        
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout_rate))
    
    model.add(Flatten())
    
    model.add(Dense(mid_units))
    model.add(Activation(activation))
    model.add(Dense(10))
    model.add(Activation('softmax'))
    
    model.compile(loss="categorical_crossentropy", optimizer="adadelta", metrics=["accuracy"])
    return model

In [179]:

x_train.shape

(60000, 28, 28, 1)

In [ ]:
my_classifier = KerasClassifier(make_model, batch_size=128)
validator = GridSearchCV(my_classifier,
                         param_grid={
                                     'num_layer' : [3, 5],
                                     'mid_units' : [300, 500, 10] ,
                                     'num_filters' : [(str(i), '16', '128', '16') for i in range(5)],
                                     'activation' : ["relu", "sigmoid"],
                                     'dropout_rate' : [0.0, 1.0],
                                     'optimizer':["sgd", "adam", "rmsprop"]},
                         scoring="neg_log_loss", 
                         n_jobs=1)
    
validator.fit(x_train, y_train)

print('The parameters of the best model are: ')
print(validator.best_params_)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


245/375 [==================>...........] - ETA: 1:12 - loss: 2.3024 - accuracy: 0.1012

In [ ]:
'best_model = validator.best_estimator_.model
metric_names = best_model.metrics_names
metric_values = best_model.evaluate(x_test, y_test)
for metric, value in zip(metric_names, metric_values):
    print(metric, ': ', value)

In [ ]:
import os
yaml_string = best_model.to_yaml()
open(os.path.join('opt/app/','keras_gridsearch.yaml'), 'w').write(yaml_string)
print('save weights')
best_model.save_weights(os.path.join('opt/app/','keras_gridsearch.hdf5'))